In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import re
from difflib import SequenceMatcher
from rapidfuzz import fuzz, process
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
from contextlib import contextmanager
import time
from joblib import Parallel, delayed
from multiprocessing import Pool, cpu_count
import math
from collections import defaultdict
import gc
import pyarrow as pa
import pyarrow.parquet as pq


# Set File Path and Prefix

In [2]:
# Define the file path and the prefix 'GER' for the CSV files
file_path = os.path.expanduser("~/Documents/IMT-Lucca/Policy-Learning/StateAid") # Replace with the actual path to your folder
cleaning_path = os.path.join(file_path, 'StateAid_Cleaning')
data_path = os.path.join(file_path, 'Data/GER')
prefix = 'Raw_GER'

In [14]:
raw_path = os.path.join(data_path, 'Raw')
print(f"Raw path: {raw_path}")

Raw path: /var/home/econmeownist/Documents/IMT-Lucca/Policy-Learning/StateAid/Data/GER/Raw


In [97]:
# The raw data files from the state aid register have the prefix 'GER' followed by the year
# List all files in the directory with the specified prefix
csv_files = [f for f in os.listdir(raw_path) if f.startswith(prefix) and f.endswith('.csv')]

# Display the list of CSV files to be merged
csv_files

['Raw_GER_2016.csv',
 'Raw_GER_2017.csv',
 'Raw_GER_2018.csv',
 'Raw_GER_2019.csv',
 'Raw_GER_2020.csv',
 'Raw_GER_2021.csv',
 'Raw_GER_2022.csv',
 'Raw_GER_2023-25.csv']

# List and Read CSV Files
List all CSV files in the directory with the specified prefix and read them into pandas DataFrames.

In [ ]:
# Read each CSV file into a pandas DataFrame and store them in a list
dataframes = [pd.read_csv(os.path.join(raw_path, file)) for file in csv_files]

# Display the first few rows of each DataFrame to verify the contents
#for df in dataframes:
    #display(df.head())

/tmp/ipykernel_43744/2748973902.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = [pd.read_csv(os.path.join(raw_path, file)) for file in csv_files]
/tmp/ipykernel_43744/2748973902.py:2: DtypeWarning: Columns (13,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = [pd.read_csv(os.path.join(raw_path, file)) for file in csv_files]


,Country,Another Beneficiary Member State,Aid Measure Title,Aid Measure Title [EN],SA.Number,Ref-no.,National ID,Name of the beneficiary,Name of the beneficiary [EN],Beneficiary Type,...,"Nominal Amount, expressed as full amount","Aid element, expressed as full amount",Currency,Date of granting,Granting Authority Name,Granting Authority Name [EN],Published Date,Entrusted Entity,Financial Intermediaries,Third country outside of the EU
0,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10115165,HRB 502044,Th�ringer Energie AG,Technically speaking Ringer Energie AG,Only large enterprises,...,NaN,4162129,EUR,06/12/2016,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,05/10/2018,NaN,NaN,NaN
1,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10115132,HRB 7173,Mainova AG,Mainova AG,Only large enterprises,...,NaN,594642,EUR,05/12/2016,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,05/10/2018,NaN,NaN,NaN
2,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10115133,HRB 7173,Mainova AG,Mainova AG,Only large enterprises,...,NaN,2754961,EUR,28/11/2016,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,05/10/2018,NaN,NaN,NaN
3,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10115135,HRB 101879,Michelin Reifenwerke AG & Co. KGaA,Michelin Tyreswerke AG & Co. KGaA,Only large enterprises,...,NaN,656028,EUR,01/09/2016,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,05/10/2018,NaN,NaN,NaN
4,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10115137,HRA 7828,Molkerei Meggle Wasserburg GmbH & Co. KG,﻿Molkerei Megatle Wasserburg GmbH & Co. kg,Only large enterprises,...,NaN,2017152,EUR,30/11/2016,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,05/10/2018,NaN,NaN,NaN


,Country,Another Beneficiary Member State,Aid Measure Title,Aid Measure Title [EN],SA.Number,Ref-no.,National ID,Name of the beneficiary,Name of the beneficiary [EN],Beneficiary Type,...,"Nominal Amount, expressed as full amount","Aid element, expressed as full amount",Currency,Date of granting,Granting Authority Name,Granting Authority Name [EN],Published Date,Entrusted Entity,Financial Intermediaries,Third country outside of the EU
0,Germany,Germany,DE - Support of renewable electricity and redu...,﻿De — Support of renewable electricity as a re...,SA.33995,TM-10116030,HRA 6236 FL,Wöhrden DK GmbH & Co. KG,﻿Wöhrenden DK GmbH & Co. kg,Small and medium-sized entreprises,...,NaN,574191,EUR,31/07/2017,Übertragungsnetzbetreiber (50Hertz Transmissio...,Transmission System Operator (50Hertz Transmis...,15/10/2018,NaN,NaN,NaN
1,Germany,Germany,DE - Support of renewable electricity and redu...,﻿De — Support of renewable electricity as a re...,SA.33995,TM-10116140,DE284660287,WP Sachsen-Anhalt Süd Zwölf GmbH & Co.KG,﻿Saxony-Anhalt South 12 GmbH & Co. kg,Small and medium-sized entreprises,...,NaN,509034.65,EUR,31/07/2017,Übertragungsnetzbetreiber (50Hertz Transmissio...,Transmission System Operator (50Hertz Transmis...,15/10/2018,NaN,NaN,NaN
2,Germany,Germany,DE - Support of renewable electricity and redu...,﻿De — Support of renewable electricity as a re...,SA.33995,TM-10116141,DE284660295,WP Sachsen-Anhalt Süd Dreizehn GmbH & Co.KG,﻿Saxony-Anhalt South Thirteen GmbH & Co. kg,Small and medium-sized entreprises,...,NaN,549141.12,EUR,31/07/2017,Übertragungsnetzbetreiber (50Hertz Transmissio...,Transmission System Operator (50Hertz Transmis...,15/10/2018,NaN,NaN,NaN
3,Germany,Germany,DE - Support of renewable electricity and redu...,﻿De — Support of renewable electricity as a re...,SA.33995,TM-10116142,15/292/29373,Solarpark Ziegeleiweg Kremmen GmbH,﻿Solarpark brick and bewegen route GmbH,Small and medium-sized entreprises,...,NaN,1079295.04,EUR,31/07/2017,Übertragungsnetzbetreiber (50Hertz Transmissio...,Transmission System Operator (50Hertz Transmis...,15/10/2018,NaN,NaN,NaN
4,Germany,Germany,DE - Support of renewable electricity and redu...,﻿De — Support of renewable electricity as a re...,SA.33995,TM-10116143,15/292/08006,Solarpark Rapshagen GmbH,﻿Oilseed rape Hraphagen GmbH,Small and medium-sized entreprises,...,NaN,1676784.33,EUR,31/07/2017,Übertragungsnetzbetreiber (50Hertz Transmissio...,Transmission System Operator (50Hertz Transmis...,15/10/2018,NaN,NaN,NaN


,Country,Another Beneficiary Member State,Aid Measure Title,Aid Measure Title [EN],SA.Number,Ref-no.,National ID,Name of the beneficiary,Name of the beneficiary [EN],Beneficiary Type,...,"Nominal Amount, expressed as full amount","Aid element, expressed as full amount",Currency,Date of granting,Granting Authority Name,Granting Authority Name [EN],Published Date,Entrusted Entity,Financial Intermediaries,Third country outside of the EU
0,Germany,Germany,Richtlinie zur Förderung von Forschungsvorhabe...,﻿Directive promoting research on improving exp...,SA.50601,TM-10109895,HRB 680693 Ulm,WITTENSTEIN cyber motor GmbH,﻿WITTENSTEIN Cyber Motor GmbH,Only large enterprises,...,NaN,672924,EUR,24/07/2018,Bundesministerium für Bildung und Forschung,Federal Ministry of Education and Research,25/09/2018,NaN,NaN,NaN
1,Germany,Germany,Photonics Research. R&D-scheme. Germany,Photonics Research. R & D scheme. Germany,SA.32795,TM-10114424,HRB 280355 AG Freiburg im Breisgau,SICK AG,SICK AG,Only large enterprises,...,NaN,725930,EUR,24/09/2018,Bundesministerium für Bildung und Forschung,Federal Ministry of Education and Research,01/10/2018,NaN,NaN,NaN
2,Germany,Germany,Bund: National Framework on granting State aid...,Federal authorities: National Framework on gra...,SA.40354,TM-10109807,xxx,Mader Tobias,﻿Mader Tobias,Small and medium-sized entreprises,...,NaN,86342.6,EUR,04/05/2018,Landratsamt Bodenseekreis,﻿Bodenseekreis district authority,04/10/2018,NaN,NaN,NaN
3,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10115166,HRA 5243,T�nnies Lebensmittel GmbH & Co. KG,﻿T kt nnices Food GmbH & Co. kg,Only large enterprises,...,NaN,1209918,EUR,12/04/2018,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,05/10/2018,NaN,NaN,NaN
4,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10115169,HRB 10216,Urbana Energiedienste GmbH,﻿Urbana Energiedienste GmbH,Only large enterprises,...,NaN,1036127,EUR,16/01/2018,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,05/10/2018,NaN,NaN,NaN


,Country,Another Beneficiary Member State,Aid Measure Title,Aid Measure Title [EN],SA.Number,Ref-no.,National ID,Name of the beneficiary,Name of the beneficiary [EN],Beneficiary Type,...,"Nominal Amount, expressed as full amount","Aid element, expressed as full amount",Currency,Date of granting,Granting Authority Name,Granting Authority Name [EN],Published Date,Entrusted Entity,Financial Intermediaries,Third country outside of the EU
0,Germany,Germany,Seedfonds Niedersachsen - NSeed [NI],Seed Fund Lower Saxony — NSeed [NI],SA.102217,TM-11882783,DE305701255,B&S Media GmbH,B &amp; S Media GmbH,Small and medium-sized entreprises,...,NaN,500000,EUR,15/08/2019,NBank,NBank,13/09/2023,NBank,NaN,NaN
1,Germany,Germany,Vorschrift der Stadt Freiburg über die Förderu...,Rule of the City of Freiburg on the promotion ...,SA.47123,TM-10096769,HRB 952 Registerger. Freiburg,Freiburger Verkehrs AG,Freundurger Transport AG,Only large enterprises,...,NaN,938820.29,EUR,07/10/2019,Stadt Freiburg,City of Freiburg,10/04/2018,NaN,NaN,NaN
2,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10221354,HRB 560,Stadtwerke Aachen AG,NaN,Only large enterprises,...,NaN,547027,EUR,16/09/2019,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,16/11/2020,NaN,NaN,NaN
3,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10221358,HRB 501,Stadtwerke Burg GmbH,NaN,Small and medium-sized entreprises,...,NaN,678298,EUR,29/08/2019,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,16/11/2020,NaN,NaN,NaN
4,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10221454,GnR 375 FL,Energiegenossenschaft F�hr eG,NaN,Small and medium-sized entreprises,...,NaN,663923,EUR,05/08/2019,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,16/11/2020,NaN,NaN,NaN


,Country,Another Beneficiary Member State,Aid Measure Title,Aid Measure Title [EN],SA.Number,Ref-no.,National ID,Name of the beneficiary,Name of the beneficiary [EN],Beneficiary Type,...,"Nominal Amount, expressed as full amount","Aid element, expressed as full amount",Currency,Date of granting,Granting Authority Name,Granting Authority Name [EN],Published Date,Entrusted Entity,Financial Intermediaries,Third country outside of the EU
0,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10221353,GnR 720099,BWO Energie eG,NaN,Small and medium-sized entreprises,...,NaN,688921,EUR,30/07/2020,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,16/11/2020,NaN,NaN,NaN
1,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10221355,HRA 105947,SWM Infrastruktur GmbH & Co. KG,NaN,Only large enterprises,...,NaN,654115,EUR,23/07/2020,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,16/11/2020,NaN,NaN,NaN
2,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10221356,HRB 2847,Stadtwerke Ingolstadt Energie GmbH,NaN,Only large enterprises,...,NaN,1097253,EUR,10/09/2020,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,16/11/2020,NaN,NaN,NaN
3,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10221456,HRB 3896,Energieservice Westfalen Weser GmbH,NaN,Only large enterprises,...,NaN,537066,EUR,14/07/2020,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,16/11/2020,NaN,NaN,NaN
4,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10221458,HRB 3420,Stadtwerke Detmold GmbH,NaN,Small and medium-sized entreprises,...,NaN,552565,EUR,25/08/2020,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,16/11/2020,NaN,NaN,NaN


,Country,Another Beneficiary Member State,Aid Measure Title,Aid Measure Title [EN],SA.Number,Ref-no.,National ID,Name of the beneficiary,Name of the beneficiary [EN],Beneficiary Type,...,"Nominal Amount, expressed as full amount","Aid element, expressed as full amount",Currency,Date of granting,Granting Authority Name,Granting Authority Name [EN],Published Date,Entrusted Entity,Financial Intermediaries,Third country outside of the EU
0,Germany,Germany,Seedfonds Niedersachsen - NSeed [NI],Seed Fund Lower Saxony — NSeed [NI],SA.102217,TM-11882792,HRB 209437,DREEMS e-mobilities GmbH,DREEMS e-Mobilities GmbH,Small and medium-sized entreprises,...,NaN,500000,EUR,29/06/2021,NBank,NBank,13/09/2023,NBank,NaN,NaN
1,Germany,Germany,Richtlinie über die Gewährung von Zuwendungen ...,Guidelines on the granting of grants for the s...,SA.58627,TM-11882822,DE 811 231 08,Technische Universität Berlin Fachgebiet Flugm...,"Technical University of Berlin, Department of ...",Only large enterprises,...,NaN,509300,EUR,09/03/2021,NBank,NBank,13/09/2023,NaN,NaN,NaN
2,Germany,Germany,Richtlinie Digitalisierung im Verkehr [NI],Digital Transport Directive [NI],SA.60329,TM-11884300,HRB 9748,Oecon Products & Services GmbH,NaN,Small and medium-sized entreprises,...,NaN,1206000,EUR,18/03/2021,NBank,NBank,13/09/2023,NaN,NaN,NaN
3,Germany,Germany,Implementierung eines wetterunabhängigen und h...,Implementation of a weather-independent and hi...,SA.64477,TM-11884027,HRB 53050,TÜV Rheinland InterTraffic GmbH,NaN,Only large enterprises,...,NaN,893123.89,EUR,23/02/2021,Bundesanstalt für Verwaltungsdienstleistungen,Federal government services,13/09/2023,NaN,NaN,NaN
4,Germany,Germany,Implementierung eines wetterunabhängigen und h...,Implementation of a weather-independent and hi...,SA.64478,TM-11884028,HRB 192096 B,ViaVan GmbH,ViaVan GmbH,Only large enterprises,...,NaN,1013563.26,EUR,23/02/2021,Bundesanstalt für Verwaltungsdienstleistungen,Federal government services,13/09/2023,NaN,NaN,NaN


,Country,Another Beneficiary Member State,Aid Measure Title,Aid Measure Title [EN],SA.Number,Ref-no.,National ID,Name of the beneficiary,Name of the beneficiary [EN],Beneficiary Type,...,"Nominal Amount, expressed as full amount","Aid element, expressed as full amount",Currency,Date of granting,Granting Authority Name,Granting Authority Name [EN],Published Date,Entrusted Entity,Financial Intermediaries,Third country outside of the EU
0,Germany,Germany,Aufbau eines Innovationsclusters für Künstlich...,Building an innovation cluster on Artificial I...,SA.103715,TM-11882798,HRB 216532,CDC - Coppenrath Digital Innovation Cluster GmbH,CDC – Coppenrath Digital Innovation Cluster GmbH,Small and medium-sized entreprises,...,NaN,4909248,EUR,30/06/2022,NBank,NBank,13/09/2023,NaN,NaN,NaN
1,Germany,Germany,Germany - Amendment of the Federal aid scheme ...,Germany – Amendment of the Federal aid scheme ...,SA.103346,TM-11774519,DE 310625283,Spedition Artur Kossack Inhaberin Erna Dix,Freight forwarding Artur Kossack Owner Erna Dix,Small and medium-sized entreprises,...,NaN,1206800,EUR,19/12/2022,Bundesamt für Güterverkehr,Federal Office for the Transport of Goods,29/08/2023,NaN,NaN,NaN
2,Germany,Germany,Germany - Amendment of the Federal aid scheme ...,Germany – Amendment of the Federal aid scheme ...,SA.103346,TM-11774560,HRB 2854,Strobel Quarzsand GmbH,Strobel Quarzsand GmbH,Small and medium-sized entreprises,...,NaN,384969.6,EUR,15/12/2022,Bundesamt für Güterverkehr,Federal Office for the Transport of Goods,29/08/2023,NaN,NaN,NaN
3,Germany,Germany,Germany - Amendment of the Federal aid scheme ...,Germany – Amendment of the Federal aid scheme ...,SA.103346,TM-11774509,DE251273055,Gruber Logistics GmbH,Gruber Logistics GmbH,Only large enterprises,...,NaN,5277080,EUR,15/12/2022,Bundesamt für Güterverkehr,Federal Office for the Transport of Goods,29/08/2023,NaN,NaN,NaN
4,Germany,Germany,"Bund-Länder-Gemeinschaftsaufgabe ""Verbesserung...",Federal Government/Länder joint task “Improvem...,SA.101541,TM-11772408,DE317051578,Heise GmbH,Heise GmbH,Small and medium-sized entreprises,...,NaN,601060,EUR,09/12/2022,NBank,NBank,01/09/2023,NaN,NaN,NaN


,Country,Another Beneficiary Member State,Aid Measure Title,Aid Measure Title [EN],SA.Number,Ref-no.,National ID,Name of the beneficiary,Name of the beneficiary [EN],Beneficiary Type,...,"Nominal Amount, expressed as full amount","Aid element, expressed as full amount",Currency,Date of granting,Granting Authority Name,Granting Authority Name [EN],Published Date,Entrusted Entity,Financial Intermediaries,Third country outside of the EU
0,Germany,Germany,Hessen - Digitalisation of agriculture,Hessen — Digitalisation of agriculture,SA.60603,TM-11881445,PI 856435,Michael Dörr,Michael Dörr,Small and medium-sized entreprises,...,NaN,34924,EUR,08/05/2023,Regierungspräsidium Gießen,Regierungspräsidium Gießen,05/09/2023,NaN,NaN,NaN
1,Germany,Germany,Hessen - Digitalisation of agriculture,Hessen — Digitalisation of agriculture,SA.60603,TM-11881452,PI 9019611,Martin Kuhn,Martin Kuhn,Small and medium-sized entreprises,...,NaN,25252,EUR,16/05/2023,Regierungspräsidium Gießen,Regierungspräsidium Gießen,05/09/2023,NaN,NaN,NaN
2,Germany,Germany,AIAMO - Artificial Intelligence and Mobility/K...,AIAMO – Artificial Intelligence and Mobility [...,SA.108709,TM-11882873,HRB 14000,Robert Bosch Gesellschaft mit beschränkter Haf...,NaN,Only large enterprises,...,NaN,1076027.71,EUR,29/06/2023,Bundesanstalt für Verwaltungsdienstleistungen,Federal government services,13/09/2023,NaN,NaN,NaN
3,Germany,Germany,Germany - Evaluation plan for the block exempt...,Germany — Evaluation plan for the block exempt...,SA.56245,TM-11881220,HRB 196674,Shore GmbH,Shore GmbH,Small and medium-sized entreprises,...,NaN,"> 500,000 - 1,000,000",EUR,13/09/2023,Finanzamt München Abt. 3,"Munich Tax Office, Department 3",13/09/2023,NaN,NaN,NaN
4,Germany,Germany,AIAMO - Artificial Intelligence and Mobility/K...,AIAMO – Artificial Intelligence and Mobility [...,SA.108704,TM-11882872,DE 147 839 419,Forschungsinstitut für Kraftfahrwesen und Fahr...,Research Institute for Motor Vehicles and Moto...,Small and medium-sized entreprises,...,NaN,819243.87,EUR,29/06/2023,Bundesanstalt für Verwaltungsdienstleistungen,Federal government services,13/09/2023,NaN,NaN,NaN


# Merge DataFrames
Merge all the DataFrames into a single DataFrame.

In [ ]:
# Merge all the DataFrames into a single DataFrame
merged_df = pd.concat(dataframes, ignore_index=True)

# Display the first few rows of the merged DataFrame to verify the contents
#display(merged_df.head())

,Country,Another Beneficiary Member State,Aid Measure Title,Aid Measure Title [EN],SA.Number,Ref-no.,National ID,Name of the beneficiary,Name of the beneficiary [EN],Beneficiary Type,...,"Nominal Amount, expressed as full amount","Aid element, expressed as full amount",Currency,Date of granting,Granting Authority Name,Granting Authority Name [EN],Published Date,Entrusted Entity,Financial Intermediaries,Third country outside of the EU
0,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10115165,HRB 502044,Th�ringer Energie AG,Technically speaking Ringer Energie AG,Only large enterprises,...,NaN,4162129,EUR,06/12/2016,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,05/10/2018,NaN,NaN,NaN
1,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10115132,HRB 7173,Mainova AG,Mainova AG,Only large enterprises,...,NaN,594642,EUR,05/12/2016,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,05/10/2018,NaN,NaN,NaN
2,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10115133,HRB 7173,Mainova AG,Mainova AG,Only large enterprises,...,NaN,2754961,EUR,28/11/2016,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,05/10/2018,NaN,NaN,NaN
3,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10115135,HRB 101879,Michelin Reifenwerke AG & Co. KGaA,Michelin Tyreswerke AG & Co. KGaA,Only large enterprises,...,NaN,656028,EUR,01/09/2016,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,05/10/2018,NaN,NaN,NaN
4,Germany,Germany,Reform of support for cogeneration in Germany,Reform of support for cogeneration in Germany,SA.42393,TM-10115137,HRA 7828,Molkerei Meggle Wasserburg GmbH & Co. KG,﻿Molkerei Megatle Wasserburg GmbH & Co. kg,Only large enterprises,...,NaN,2017152,EUR,30/11/2016,Bundesamt für Wirtschaft und Ausfuhrkontrolle,Federal Office of Economics and Export Control,05/10/2018,NaN,NaN,NaN


# Save Merged DataFrame
Save the merged DataFrame to a new CSV file.

In [ ]:
# Save the merged DataFrame to a new CSV file
output_file_path = os.path.join(data_path, 'Raw/merged_GER.csv')  # Replace with the desired output file path
merged_df.to_csv(output_file_path, index=False)

# Get NID from StateAid Register

Get National ID from State Aid data to get search on orbis

In [ ]:
df_filtered = pd.read_csv(os.path.join(data_path, 'Raw/merged_GER.csv'))

/tmp/ipykernel_51605/1693412780.py:1: DtypeWarning: Columns (13,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_filtered = pd.read_csv(os.path.join(file_path, 'Raw/merged_GER.csv'))


In [38]:
# Keep only the 'National ID' columns
df_filtered = df_filtered[['National ID']]

In [39]:
# Filter rows that contain "Ust-IdNr" in the "National ID" column, case-insensitive
df_add_ustid_trade = df_filtered[df_filtered['National ID'].str.contains('ust', case=False, na=False) & df_filtered['National ID'].str.contains('id', case=False, na=False)]
# Filter rows that contain "Amtsgericht" in the "National ID" column, case-insensitive
df_amtsgericht = df_filtered[df_filtered['National ID'].str.contains('Amtsgericht', case=False, na=False)]

# Concatenate the filtered rows to df_add_ustid_trade
df_add_ustid_trade = pd.concat([df_add_ustid_trade, df_amtsgericht], ignore_index=True)

df_add_VR_trade = df_filtered.copy()

In [ ]:
# Define the clean_ust_id function
def clean_ust_id(national_id):
    # Match the sequence 'DE' followed by digits, ignoring spaces and stopping at non-digit characters
    match = re.search(r'DE\s*\d{1,3}\s*\d{1,3}\s*\d{1,3}', national_id)
    if match:
        # Remove all spaces from the matched string
        return re.sub(r'\s+', '', match.group(0))
    return national_id  # Return the original national_id if no match is found

# Define a function to clean 'National ID' for Trade ID
def clean_amtsgericht(national_id):
    # Match the sequence 'HRA', 'HRB', 'HR A', 'HR B', or 'VR' followed by digits, ignoring spaces and special characters
    match = re.search(r'(HRA|HRB|HR\s*A|HR\s*B|VR)\s*\d+', national_id, re.IGNORECASE)
    if match:
        # Remove all spaces and special characters from the matched string
        return re.sub(r'[\s_()]+', '', match.group(0))
    return national_id  # Return the original national_id if no match is found

# Ensure 'National ID' column is of string type and handle NaN values
df_add_ustid_trade['National ID'] = df_add_ustid_trade['National ID'].fillna('').astype(str)
df_add_VR_trade['National ID'] = df_add_VR_trade['National ID'].fillna('').astype(str)

# Apply the cleaning functions to the appropriate rows
df_add_ustid_trade['National ID'] = df_add_ustid_trade.apply(
    lambda row: clean_ust_id(row['National ID']) if re.search(r'ust', row['National ID'], re.IGNORECASE) and re.search(r'id', row['National ID'], re.IGNORECASE) else row['National ID'],
    axis=1
)

df_add_ustid_trade['National ID'] = df_add_ustid_trade.apply(
    lambda row: clean_amtsgericht(row['National ID']) if re.search(r'Amtsgericht', row['National ID'], re.IGNORECASE) else row['National ID'],
    axis=1
)

df_add_VR_trade['National ID'] = df_add_VR_trade.apply(
    lambda row: clean_amtsgericht(row['National ID']),
    axis=1
)

df_nid = pd.concat([df_filtered, df_add_ustid_trade, df_add_VR_trade], ignore_index=True)

In [ ]:
# Export
output_ustid_trade_path = os.path.join(data_path, 'NID_for_Orbis/NID_GER.csv')
df_nid.to_csv(output_ustid_trade_path, index=False)

# Clean Trade number files from Orbis

In [3]:
# Function to extract VAT and Tax numbers
def extract_numbers(vat_tax_number):
    if isinstance(vat_tax_number, str):
        parts = vat_tax_number.split('\n')
        if len(parts) == 2:
            return parts[0], parts[1]  # VAT, Tax
        elif len(parts) == 1:
            if 'DE' in parts[0]:  # Check if it contains 'DE'
                return parts[0], None  # VAT, No Tax
            else:
                return None, parts[0]  # No VAT, Tax
    return None, None  # Handle non-string or unexpected formats


In [ ]:
# Datafiles from Orbis have prefix 'GER_TradeNo_'
#prefix_trade = 'GER_TradeNo_'
prefix_trade = 'GER_Trade'

trade_path = os.path.join(data_path, 'Raw_from_Orbis')

# List all Excel files in the directory with the specified prefix
xlsx_files = [f for f in os.listdir(trade_path) if f.startswith(prefix_trade) and f.endswith('.xlsx')]

# Process each Excel file
for xlsx_file in xlsx_files:
    # Load the Excel file into a DataFrame
    df_xlsx = pd.read_excel(os.path.join(trade_path, xlsx_file), sheet_name='Results')
    
    #Rename the first column to 'Index'
    df_xlsx.rename(columns={df_xlsx.columns[0]: 'Index'}, inplace=True)
    
    # Create a new column 'Tax number' and set it to NaN
    df_xlsx['Tax number'] = np.nan

    # Apply the extraction function
    df_xlsx[['VAT number', 'Tax number']] = df_xlsx['VAT/Tax number'].apply(extract_numbers).tolist()

    # Drop the original VAT/Tax number column
    df_xlsx = df_xlsx.drop('VAT/Tax number', axis=1)

    # Reorder the last three columns: VAT number, Trade register number, Tax number
    columns_order = df_xlsx.columns.tolist()  # Get the current column order as a list
    last_three_columns = ['VAT number', 'Trade register number', 'Tax number']  # Specify the desired order for the last three columns

    # Remove the last three columns from the current order and append them in the desired order
    columns_order = [col for col in columns_order if col not in last_three_columns] + last_three_columns

    # Reorder the DataFrame columns
    df_xlsx = df_xlsx[columns_order]
    
    # Define the output Excel file path
    output_file_path = os.path.join(data_path, 'Cleaned_from_Orbis',f"filtered_{xlsx_file}")
    
    # Save the filtered DataFrame to a new Excel file
    df_xlsx.to_excel(output_file_path, index=False)
    
    print(f"Filtered Excel file has been saved at: {output_file_path}")

Filtered Excel file has been saved at: /var/home/econmeownist/Documents/IMT-Lucca/Policy-Learning/StateAid/StateAid_Cleaning/Cleaned_from_Orbis/filtered_GER_TradeTaxVAT_150001-300000.xlsx
Filtered Excel file has been saved at: /var/home/econmeownist/Documents/IMT-Lucca/Policy-Learning/StateAid/StateAid_Cleaning/Cleaned_from_Orbis/filtered_GER_TradeTaxVAT_300001-450000.xlsx
Filtered Excel file has been saved at: /var/home/econmeownist/Documents/IMT-Lucca/Policy-Learning/StateAid/StateAid_Cleaning/Cleaned_from_Orbis/filtered_GER_TradeTaxVAT_450001-600000.xlsx
Filtered Excel file has been saved at: /var/home/econmeownist/Documents/IMT-Lucca/Policy-Learning/StateAid/StateAid_Cleaning/Cleaned_from_Orbis/filtered_GER_TradeTaxVAT_600001-750000.xlsx
Filtered Excel file has been saved at: /var/home/econmeownist/Documents/IMT-Lucca/Policy-Learning/StateAid/StateAid_Cleaning/Cleaned_from_Orbis/filtered_GER_TradeTaxVAT_750001-856205.xlsx
Filtered Excel file has been saved at: /var/home/econmeownis

# Clean Tax number file from Orbis

In [ ]:
# Define the Excel file path
excel_file_path = os.path.join(data_path, 'Raw_from_Orbis/GER_TaxNo.xlsx')  # Replace with your Excel file path

# Load the Excel file into a DataFrame (assuming the sheet name is 'Results')
df = pd.read_excel(excel_file_path, sheet_name='Results')

# Create a new column 'Tax number' and initialize it with NaN
df['Tax number'] = np.nan

# Apply the extraction function
df[['VAT number', 'Tax number']] = df['VAT/Tax number'].apply(extract_numbers).tolist()

# Drop the original VAT/Tax number column
df = df.drop('VAT/Tax number', axis=1)

# Reorder the last three columns: VAT number, Trade register number, Tax number
columns_order = df.columns.tolist()  # Get the current column order as a list
last_three_columns = ['VAT number', 'Trade register number', 'Tax number']  # Specify the desired order for the last three columns

# Remove the last three columns from the current order and append them in the desired order
columns_order = [col for col in columns_order if col not in last_three_columns] + last_three_columns

# Reorder the DataFrame columns
df = df[columns_order]

# Define the output Excel file path
output_file_path = os.path.join(data_path, 'Cleaned_from_Orbis', 'filtered_GER_TaxNo.xlsx')  # Replace with your desired output Excel file path

# Save the filtered DataFrame to a new Excel file
df.to_excel(output_file_path, index=False)

print(f"Filtered Excel file has been saved at: {output_file_path}")

# Clean VAT number


In [ ]:
# Define the Excel file path
excel_file_path = os.path.join(data_path, 'Raw_from_Orbis/GER_Ust-VAT.xlsx')  # Replace with your Excel file 

# Load the Excel file into a DataFrame (assuming the sheet name is 'Results')
df = pd.read_excel(excel_file_path, sheet_name='Results')

# Create a new column 'Tax number' and initialize it with NaN
df['Tax number'] = np.nan

# Apply the extraction function
df[['VAT number', 'Tax number']] = df['VAT/Tax number'].apply(extract_numbers).tolist()

# Drop the original VAT/Tax number column
df = df.drop('VAT/Tax number', axis=1)

# Reorder the last three columns: VAT number, Trade register number, Tax number
columns_order = df.columns.tolist()  # Get the current column order as a list
last_three_columns = ['VAT number', 'Trade register number', 'Tax number']  # Specify the desired order for the last three columns

# Remove the last three columns from the current order and append them in the desired order
columns_order = [col for col in columns_order if col not in last_three_columns] + last_three_columns

# Reorder the DataFrame columns
df = df[columns_order]

# Define the output Excel file path
output_file_path = os.path.join(data_path, 'Cleaned_from_Orbis', 'filtered_GER_Ust-VAT.xlsx')  # Replace with your desired output Excel file path

# Save the filtered DataFrame to a new Excel file
df.to_excel(output_file_path, index=False)

print(f"Filtered Excel file has been saved at: {output_file_path}")

Filtered Excel file has been saved at: /var/home/econmeownist/Documents/IMT-Lucca/Policy-Learning/StateAid/StateAid_Cleaning/Cleaned_from_Orbis/filtered_GER_Ust-VAT.xlsx


# Merge all national id types

In [ ]:
prefix_filtered = 'filtered_GER_'

cleaned_path = os.path.join(data_path, 'Cleaned_from_Orbis')

# List all Excel files in the directory with the specified prefix
xlsx_files = [f for f in os.listdir(cleaned_path) if f.startswith(prefix_filtered) and f.endswith('.xlsx')]

# Initialize an empty list to store DataFrames
dataframes = []

# Process each Excel file
for xlsx_file in xlsx_files:
    # Load the Excel file
    excel_path = os.path.join(cleaned_path, xlsx_file)
    xls = pd.ExcelFile(excel_path)
    
    # Check if the "Results" sheet exists, otherwise use the first sheet
    sheet_name = 'Results' if 'Results' in xls.sheet_names else xls.sheet_names[0]
    
    # Load the sheet into a DataFrame
    df = pd.read_excel(excel_path, sheet_name=sheet_name)
    
    #Remove the first column
    df = df.iloc[:, 1:]
    
    ## Remove the dash and the numbers following it in the 'BvD ID number'
    df['BvD ID number'] = df['BvD ID number'].str.replace(r'-\d+', '', regex=True)

    # Remove duplicates based on the cleaned 'BvD ID number'
    df = df.drop_duplicates(subset=['BvD ID number'], keep='first')
    

    # Ensure consistent data types
    df = df.astype(str)
    
    # Append the DataFrame to the list
    dataframes.append(df)

# Merge all the DataFrames into a single DataFrame
merged_df = pd.concat(dataframes, ignore_index=True)

# Replace 'nan' string with actual NaN in the specified columns
merged_df[['Tax number', 'Trade register number', 'VAT number']] = merged_df[['Tax number', 'Trade register number', 'VAT number']].replace('nan', np.nan)

# Define the output Excel file path
output_file_path = os.path.join(data_path, 'Cleaned_from_Orbis' ,'found_NID_GER.xlsx')  # Replace with your desired output file path

# Save the merged DataFrame to a new Excel file
merged_df.to_excel(output_file_path, index=False)

print(f"Merged Excel file has been saved at: {output_file_path}")

Merged Excel file has been saved at: /var/home/econmeownist/Documents/IMT-Lucca/Policy-Learning/StateAid/StateAid_Cleaning/Cleaned_from_Orbis/found_NID_GER.xlsx


# Remove not-found firms from the register

In [3]:
# Define the path
csv_file_path = os.path.join(data_path, "Raw/merged_GER.csv")  # Replace with your CSV file path
xlsx_file_path = os.path.join(data_path, "Cleaned_from_Orbis/found_NID_GER.xlsx")  # Replace with your Excel file path

# Load the files
columns_to_load = ['Ref-no.','Name of the beneficiary','National ID']
df_csv = pd.read_csv(csv_file_path, usecols=columns_to_load, dtype={'National ID': str})
df_xlsx = pd.read_excel(xlsx_file_path, dtype={'VAT number': str, 'Trade register number': str, 'Tax number': str})

df_xlsx.rename(columns={'Tax number': 'taxno','Trade register number': 'tradeno', 'VAT number': 'vatno'}, inplace=True)
df_xlsx = df_xlsx.drop_duplicates(subset=['BvD ID number'], keep='first')

In [ ]:
# Convert 'National ID' to string type
df_csv['National ID'] = df_csv['National ID'].astype(str)
df_xlsx['vatno'] = df_xlsx['vatno'].astype(str)
df_xlsx['tradeno'] = df_xlsx['tradeno'].astype(str)
df_xlsx['taxno'] = df_xlsx['taxno'].astype(str)

# Create the 'nid_indic' column 
#df_csv['nid_indic'] = df_csv['National ID'].apply(lambda x: 1 if re.search(r'\b(HRA|HRB|HR A|HR B|VR)\b', x, re.IGNORECASE) else 0)

df_csv['Name of the beneficiary'] = df_csv['Name of the beneficiary'].astype(str)
df_xlsx['Company name Latin alphabet'] = df_xlsx['Company name Latin alphabet'].astype(str)

# Rename the column without assigning the result back to df_xlsx
df_xlsx.rename(columns={'Company name\nLocal Alphabet': 'Company name Local Alphabet'}, inplace=True)
df_xlsx['Company name Local Alphabet'] = df_xlsx['Company name Local Alphabet'].astype(str)

In [6]:
# Create nid columns
df_csv['nidfororbis'] = df_csv['National ID']
df_csv = df_csv.drop_duplicates(subset=['National ID', 'Name of the beneficiary'], keep='first')
df_add_VR_trade = df_csv.copy()


# Filter rows that contain "Ust-IdNr" in the "National ID" column, case-insensitive
df_add_ustid_trade = df_csv[df_csv['nidfororbis'].str.contains('ust', case=False, na=False) & df_csv['nidfororbis'].str.contains('id', case=False, na=False)]
# Filter rows that contain "Amtsgericht" in the "National ID" column, case-insensitive
df_amtsgericht = df_csv[df_csv['nidfororbis'].str.contains('Amtsgericht', case=False, na=False)]

# Concatenate the filtered rows to df_add_ustid_trade
df_add_ustid_trade = pd.concat([df_add_ustid_trade, df_amtsgericht], ignore_index=True)

# Define the clean_ust_id function
def clean_ust_id(national_id):
    # Match the sequence 'DE' followed by digits, ignoring spaces and stopping at non-digit characters
    match = re.search(r'DE\s*\d{1,3}\s*\d{1,3}\s*\d{1,3}', national_id)
    if match:
        # Remove all spaces from the matched string
        return re.sub(r'\s+', '', match.group(0))
    return national_id  # Return the original national_id if no match is found

# Define a function to clean 'National ID' for Trade ID
def clean_amtsgericht(national_id):
    # Match the sequence 'HRA', 'HRB', 'HR A', 'HR B', or 'VR' followed by digits, ignoring spaces and special characters
    match = re.search(r'(HRA|HRB|HR\s*A|HR\s*B|VR)\s*\d+', national_id, re.IGNORECASE)
    if match:
        # Remove all spaces and special characters from the matched string
        return re.sub(r'[\s_()]+', '', match.group(0))
    return national_id  # Return the original national_id if no match is found

# Apply the cleaning functions to the appropriate rows
df_add_ustid_trade['nidfororbis'] = df_add_ustid_trade.apply(
    lambda row: clean_ust_id(row['nidfororbis']) if re.search(r'ust', row['nidfororbis'], re.IGNORECASE) and re.search(r'id', row['nidfororbis'], re.IGNORECASE) else row['nidfororbis'],
    axis=1
)

df_add_ustid_trade['nidfororbis'] = df_add_ustid_trade.apply(
    lambda row: clean_amtsgericht(row['nidfororbis']) if re.search(r'Amtsgericht', row['nidfororbis'], re.IGNORECASE) else row['nidfororbis'],
    axis=1
)

df_add_VR_trade['nidfororbis'] = df_add_VR_trade.apply(
    lambda row: clean_amtsgericht(row['nidfororbis']),
    axis=1
)

df_csv = pd.concat([df_csv, df_add_VR_trade, df_add_ustid_trade], ignore_index=True)

df_unique = df_csv.drop_duplicates(subset=['National ID', 'Name of the beneficiary'], keep='last')
#df_unique = df_unique.drop_duplicates(subset=['National ID'], keep='last')
#df_unique = df_csv
df_unique = df_unique.reset_index(drop=True)

# Step 1: Locate all .txt files with the prefix "not-found" in the folder
file_pattern = os.path.join(data_path, 'Raw_from_Orbis/not-found*.txt')  # Match all files starting with "not-found"
txt_files = glob.glob(file_pattern)

# Step 2: Read and merge all .txt files
merged_ids = pd.concat(
    [
        pd.DataFrame(
            {'National ID': [line.strip() for line in open(file, 'r', encoding='utf-8') if line.strip()]}
        )
        for file in txt_files
    ]
).drop_duplicates()

# Step 3: Drop rows in df_unique where 'National ID' matches any in the merged IDs
df_unique_filtered = df_unique[~df_unique['nidfororbis'].isin(merged_ids['National ID'])]
print(df_unique_filtered.columns)

df_unique_filtered['nid'] = df_unique_filtered.apply(lambda row: clean_ust_id(row['nidfororbis']), axis=1)
df_unique_filtered['nid'] = df_unique_filtered.apply(lambda row: clean_amtsgericht(row['nidfororbis']), axis=1)

df_unique_filtered.drop(columns=['nidfororbis'], inplace=True)
df_unique_filtered = df_unique_filtered.reset_index(drop=True)

# Display the resulting DataFrame
print(df_unique_filtered)

Index(['Ref-no.', 'National ID', 'Name of the beneficiary', 'nidfororbis'], dtype='object')


/tmp/ipykernel_406363/1462588228.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_filtered['nid'] = df_unique_filtered.apply(lambda row: clean_ust_id(row['nidfororbis']), axis=1)


            Ref-no.                        National ID  \
0       TM-10115165                         HRB 502044   
1       TM-10115132                           HRB 7173   
2       TM-10115135                         HRB 101879   
3       TM-10115137                           HRA 7828   
4       TM-10115139                           HRB 1194   
...             ...                                ...   
144339  TM-12496237       HRB 14608 Amtsgericht Kassel   
144340  TM-12496608    HRB 170858; Amtsgericht Hamburg   
144341  TM-12546423         Amtsgericht Essen HRB 1433   
144342  TM-12546450  Amtsgericht Ingolstadt HRA 102780   
144343  TM-12546425        Amtsgericht Essen HRB 10740   

                         Name of the beneficiary        nid  
0                           Th�ringer Energie AG  HRB502044  
1                                     Mainova AG    HRB7173  
2             Michelin Reifenwerke AG & Co. KGaA  HRB101879  
3       Molkerei Meggle Wasserburg GmbH & Co. KG    HRA

/tmp/ipykernel_406363/1462588228.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_filtered['nid'] = df_unique_filtered.apply(lambda row: clean_amtsgericht(row['nidfororbis']), axis=1)
/tmp/ipykernel_406363/1462588228.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_filtered.drop(columns=['nidfororbis'], inplace=True)


In [7]:
# Clean the "nid" column 
df_unique_filtered['nid'] = df_unique_filtered['nid'].str.replace(r'[\s()/]', '', regex=True)
df_xlsx['tradeno'] = df_xlsx['tradeno'].str.replace(r'[\s()/]', '', regex=True)
df_xlsx['vatno'] = df_xlsx['vatno'].str.replace(r'[\s()/]', '', regex=True)
df_xlsx['taxno'] = df_xlsx['taxno'].str.replace(r'[\s()/]', '', regex=True)


In [26]:
# Sample for testing
# Create a random sample from df_csv
sample_size = 1000  # Define the sample size
df_sample = df_unique_filtered.sample(n=sample_size, random_state=42)

In [9]:
df_sample_test = df_sample[df_sample['Name of the beneficiary'] == 'Tucan.ai GmbH']
df_xlsx_test = df_xlsx[df_xlsx['Company name Local Alphabet'] == 'TUCAN.AI GMBH']

In [ ]:
# List of specific chunk indices to split further
specific_chunk_indices = [118, 215, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 
                          251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276,
                          277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289]

def preprocess_nid(nid):
    """Preprocess NID by removing special characters."""
    nid = str(nid)  # Ensure it's a string
    remove_chars = " .-()[]{}\\|,;:_"
    trans_table = str.maketrans("", "", remove_chars)
    return nid.translate(trans_table)

def preprocess_data(df_sample, df_xlsx):
    """Preprocess the data by cleaning NIDs."""
    df_sample = df_sample.copy()
    df_xlsx = df_xlsx.copy()
    df_sample['processed_nid'] = df_sample['nid'].apply(preprocess_nid)
    df_xlsx['processed_tradeno'] = df_xlsx['tradeno'].apply(preprocess_nid)
    df_xlsx['processed_vatno'] = df_xlsx['vatno'].apply(preprocess_nid)
    df_xlsx['processed_taxno'] = df_xlsx['taxno'].apply(preprocess_nid)
    return df_sample, df_xlsx

def match_chunk_unlimited(chunk_df, df_xlsx, threshold, output_dir, chunk_index):
    """Match a chunk of rows against all xlsx_nids and write results to disk in Parquet format."""
    results = []
    xlsx_tradeno = df_xlsx['processed_tradeno'].tolist()
    xlsx_vatno = df_xlsx['processed_vatno'].tolist()
    xlsx_taxno = df_xlsx['processed_taxno'].tolist()
    
    for _, sample_row in chunk_df.iterrows():
        sample_nid = sample_row['processed_nid']
        for idx in range(len(df_xlsx)):
            tradenoscore = fuzz.partial_ratio(sample_nid, xlsx_tradeno[idx])
            vatnoscore = fuzz.partial_ratio(sample_nid, xlsx_vatno[idx])
            taxnoscore = fuzz.partial_ratio(sample_nid, xlsx_taxno[idx])
            score = max(tradenoscore, vatnoscore, taxnoscore)
            if score >= threshold:
                matched_row = df_xlsx.iloc[idx]
                results.append({
                    'Ref-no.': sample_row['Ref-no.'],
                    'National ID': sample_row['National ID'],
                    'processed_nid': sample_nid,
                    'orbis_tradeno': matched_row['tradeno'],
                    'orbis_vatno': matched_row['vatno'],
                    'orbis_taxno': matched_row['taxno'],
                    'Name of the beneficiary': sample_row['Name of the beneficiary'],
                    'orbis_name': matched_row['Company name Local Alphabet'],
                    'bvd_id': matched_row['BvD ID number'],
                    'nidscore': score
                })
    
    # Write results to a temporary Parquet file
    temp_file_path = os.path.join(output_dir, f"chunk_{chunk_index}.parquet")
    pd.DataFrame(results).to_parquet(temp_file_path, index=False)
    return temp_file_path

def find_similar_nids_unlimited(df_sample, df_xlsx, threshold=70, chunk_size=50, output_dir="temp_results", n_jobs=-1):
    """Find similar NIDs and write intermediate results to disk in Parquet format."""
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Preprocess the data
    df_sample, df_xlsx = preprocess_data(df_sample, df_xlsx)

    # Split df_sample into chunks
    num_chunks = math.ceil(len(df_sample) / chunk_size)
    chunks = [df_sample.iloc[i * chunk_size:(i + 1) * chunk_size] for i in range(num_chunks)]

    # Check for already processed chunks
    processed_chunks = set(
        int(os.path.basename(file).split("_")[1].split(".")[0])  # Extract chunk index from file name
        for file in glob.glob(os.path.join(output_dir, "chunk_*.parquet"))
    )

    # Parallel processing
    temp_files = []
    with ProcessPoolExecutor() as executor:
        futures = []
        for chunk_index, chunk in enumerate(chunks):
            if chunk_index in processed_chunks:
                continue  # Skip already processed chunks
            
            # If the chunk index is in the specific list, split it further into 5 sub-chunks
            if chunk_index in specific_chunk_indices:
                sub_chunk_size = math.ceil(len(chunk) / 100)
                sub_chunks = [chunk.iloc[i * sub_chunk_size:(i + 1) * sub_chunk_size] for i in range(100)]
                for sub_chunk_index, sub_chunk in enumerate(sub_chunks):
                    futures.append(executor.submit(
                        match_chunk_unlimited, sub_chunk, df_xlsx, threshold, output_dir, f"{chunk_index}_{sub_chunk_index}"
                    ))
            else:
                futures.append(executor.submit(
                    match_chunk_unlimited, chunk, df_xlsx, threshold, output_dir, chunk_index
                ))

        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing chunks"):
            temp_files.append(future.result())

    # Combine all temporary files into a single DataFrame
    combined_results = pd.concat([pd.read_parquet(temp_file) for temp_file in glob.glob(os.path.join(output_dir, "chunk_*.parquet"))], ignore_index=True)

    # Optionally, clean up temporary files
    # Uncomment the following lines if you want to delete temporary files after combining
    # for temp_file in glob.glob(os.path.join(output_dir, "chunk_*.parquet")):
    #     os.remove(temp_file)

    return combined_results

# Usage example
output_dir = os.path.join(data_path, 'temp_parquet')  # Directory to store intermediate results
matched_df = find_similar_nids_unlimited(
    df_unique_filtered, 
    df_xlsx, 
    threshold=95, 
    chunk_size=500, 
    output_dir=output_dir, 
    n_jobs=-1
)

# Save the final results
output_path = os.path.join(data_path, 'Matching/matched_rows.parquet')
matched_df.to_parquet(output_path, index=False)

Processing chunks: 100%|██████████| 4800/4800 [3:48:03<00:00,  2.85s/it]  


In [ ]:
# Function to preprocess names by removing specific substrings
def preprocess_name(name):
    # Convert to lowercase
    name = name.lower()
    # Remove special characters
    name = re.sub(r'[^\w\s]', '', name)
    # Normalize whitespace
    name = re.sub(r'\s+', ' ', name).strip()
    patterns = [
        r'\bGMBH\b',
        r'\bGMBH & CO\b',
        r'\bGMBH & CO\. KG\b',
        r'\bKOMMANDITGESELLSCHAFT\b',
        r'\bREPORTAGEN.DE UG (HAFTUNGSBESCHRÄNKT)\b',
        r' AG$',
        r'\bAktiengesellschaft\b',
        r'\bGesellschaft mit beschränkter Haftung\b',
        r'\bsgesellschaft mbH & Co. KG\b',
        r'\bgesellschaft\b',
        r'\bgesellschaft für\b',
        r'\bmbhmbh\b'
    ]
    for pattern in patterns:
        name = re.sub(pattern, '', name, flags=re.IGNORECASE)
    return name.strip()

# Function to calculate similarity ratio between two names
def is_similar(name1, name2):
    name1 = preprocess_name(name1)
    name2 = preprocess_name(name2)
    return fuzz.WRatio(name1, name2), fuzz.partial_token_sort_ratio(name1, name2)

# Function to process a chunk of the DataFrame
def process_chunk(chunk, threshold):
    results = []
    for _, row in chunk.iterrows():
        wratio, parttoksort_ratio = is_similar(row['Name of the beneficiary'], row['orbis_name'])
        if wratio >= threshold:
            row_with_ratio = row.copy()
            row_with_ratio['wratio'] = wratio
            row_with_ratio['parttoksort_ratio'] = parttoksort_ratio
            results.append(row_with_ratio)
    return results

# Function to process a single Parquet file
def process_parquet_file(parquet_path, output_path, threshold=70, chunk_size=1000):
    # Load the Parquet file
    df_match = pd.read_parquet(parquet_path)
    
    # Split the DataFrame into chunks
    chunks = [df_match.iloc[i:i + chunk_size] for i in range(0, len(df_match), chunk_size)]
    
    results = []
    with ProcessPoolExecutor() as executor:
        futures = {executor.submit(process_chunk, chunk, threshold): chunk for chunk in chunks}
        for future in tqdm(as_completed(futures), total=len(futures), desc=f"Processing {os.path.basename(parquet_path)}"):
            results.extend(future.result())
    
    # Combine all results into a single DataFrame
    df_match_with_ratios = pd.DataFrame(results)
    
    # If df_match_with_ratios is empty, skip further processing
    if df_match_with_ratios.empty:
        print(f"Skipping filtering for {parquet_path} as no matches were found.")
        return
    
    # Group by 'Ref-no.' and find the maximum 'wratio' for each group
    max_wratio = df_match_with_ratios.groupby('Ref-no.')['wratio'].max()
    df_match_with_ratios = df_match_with_ratios.merge(
        max_wratio.rename('maxwratio'),  # Rename to avoid column name conflict
        left_on='Ref-no.',
        right_index=True,
        how='left'
    )
    
    # Filter the DataFrame to keep only rows where 'wratio' equals 'max_wratio'
    df_match_wratio_filtered = df_match_with_ratios[df_match_with_ratios['wratio'] == df_match_with_ratios['maxwratio']].copy()
    df_match_wratio_filtered.drop('maxwratio', axis=1, inplace=True)
    
    # Group by 'Ref-no.' and find the maximum 'parttoksort_ratio' for each group
    max_ptsetratio = df_match_wratio_filtered.groupby('Ref-no.')['parttoksort_ratio'].max()
    df_match_wratio_filtered = df_match_wratio_filtered.merge(
        max_ptsetratio.rename('maxptsetratio'),  # Rename to avoid column name conflict
        left_on='Ref-no.',
        right_index=True,
        how='left'
    )
    
    # Filter the DataFrame to keep only rows where 'parttoksort_ratio' equals 'max_ptsetratio'
    df_filtered2ratio = df_match_wratio_filtered[df_match_wratio_filtered['parttoksort_ratio'] == df_match_wratio_filtered['maxptsetratio']].copy()
    df_filtered2ratio.drop('maxptsetratio', axis=1, inplace=True)
    
    # Save the filtered results to a Parquet file
    df_filtered2ratio.to_parquet(output_path, index=False)

# Main processing function
def process_all_parquet_files(parquet_dir, output_dir, final_output_path, threshold=70, chunk_size=1000):
    os.makedirs(output_dir, exist_ok=True)
    
    # Get a list of already processed files in `temp_match`
    processed_files = {os.path.splitext(f)[0] for f in os.listdir(output_dir) if f.endswith('.parquet')}
    
    # Process each Parquet file
    for parquet_file in tqdm(os.listdir(parquet_dir), desc="Processing Parquet files"):
        if parquet_file.endswith('.parquet'):
            # Check if the file has already been processed
            if os.path.splitext(parquet_file)[0] in processed_files:
                print(f"Skipping already processed file: {parquet_file}")
                continue
            
            parquet_path = os.path.join(parquet_dir, parquet_file)
            output_path = os.path.join(output_dir, parquet_file)
            
            # Load the Parquet file
            df_match = pd.read_parquet(parquet_path)
            
            # Skip empty files
            if df_match.empty:
                print(f"Skipping empty file: {parquet_file}")
                continue
            
            # Process the file
            process_parquet_file(parquet_path, output_path, threshold=threshold, chunk_size=chunk_size)
    
    # Merge all filtered Parquet files into a single Parquet file
    filtered_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.parquet')]
    merged_df = pd.concat([pd.read_parquet(f) for f in filtered_files], ignore_index=True)
    merged_df.to_parquet(final_output_path, index=False)

# Directories
parquet_dir = os.path.join(data_path, 'temp_parquet')
output_dir = os.path.join(data_path, 'temp_match')
final_output_path = os.path.join(data_path, 'Matching/matched_rows_highestratio.parquet')

# Run the processing
process_all_parquet_files(parquet_dir, output_dir, final_output_path, threshold=70, chunk_size=50)

print(f"Filtered results saved to {final_output_path}")

In [ ]:
# Save the final results
def ensure_consistent_types(df):
    """
    Ensure consistent data types for all columns in the DataFrame.
    """
    for column in df.columns:
        # If the column is numeric, convert it to float and ensure no mixed types
        if pd.api.types.is_numeric_dtype(df[column]):
            df[column] = pd.to_numeric(df[column], errors='coerce').astype(float)
        # If the column is not numeric, convert it to string
        else:
            df[column] = df[column].astype(str)
    return df

def merge_and_save_parquet(parquet_dir, output_path):
    """
    Merge all Parquet files in a directory, ensure consistent types, and save as a single Parquet file.
    """
    # List all Parquet files in the directory
    parquet_files = [os.path.join(parquet_dir, f) for f in os.listdir(parquet_dir) if f.endswith('.parquet')]

    # Initialize an empty list to store DataFrames
    dataframes = []

    # Read and process each Parquet file
    for file in parquet_files:
        print(f"Processing file: {file}")
        df = pd.read_parquet(file)
        # Ensure consistent types and handle mixed types explicitly
        df = ensure_consistent_types(df)
        # Convert 'nidscore' column to float explicitly if it exists
        if 'nidscore' in df.columns:
            df['nidscore'] = pd.to_numeric(df['nidscore'], errors='coerce')
        dataframes.append(df)

    # Concatenate all DataFrames
    merged_df = pd.concat(dataframes, ignore_index=True)

    # Save the merged DataFrame as a Parquet file
    merged_df.to_parquet(output_path, index=False)
    print(f"Merged Parquet file saved at: {output_path}")

# Define the input and output paths
parquet_dir = os.path.join(data_path, 'temp_match')  # Directory containing the Parquet files
output_path = os.path.join(data_path, 'Matching/matched_rows_highestratio.parquet')  # Output file path

# Run the merging and saving process
merge_and_save_parquet(parquet_dir, output_path)

In [ ]:
df = pd.read_parquet(os.path.join(data_path, 'Matching/matched_rows_highestratio.parquet'))
df_test = df[df['parttoksort_ratio'] >= 70].copy()
df_dup = df_test[df_test.duplicated(subset=['National ID', 'Name of the beneficiary'], keep=False)].copy()
df_dup_filtered = df_dup[
    (~df_dup['National ID'].isin(['nan', 'n.a.' ,'', np.nan, None])) & 
    (~df_dup['Name of the beneficiary'].isin(['nan', '', np.nan, None]))
].copy()
df_dup_filtered = df_dup_filtered[~df_dup_filtered['Name of the beneficiary'].isin([
    'LADE GmbH',
    'Mark-E Aktiengesellschaft',
    'AUDI Aktiengesellschaft'
])]
df_dup_filtered = df_dup_filtered[df_dup_filtered['wratio'] >= 90].copy()

In [58]:
df_bvdid = df_dup_filtered['bvd_id']
# Export
df_bvdid.to_csv(os.path.join(data_path, 'Matching/bvd_id_fororbis_date.csv'), index=False, header=True)

In [ ]:
df_unchanged_bvd = pd.read_excel(os.path.join(data_path, 'Matching/Unchanged_BvDID.xlsx'), sheet_name='Results')
df_unchanged_bvd.rename(columns={'BvD ID number': 'bvd_id'}, inplace=True)
df_dup_filtered['Unchanged'] = df_dup_filtered['bvd_id'].isin(df_unchanged_bvd['bvd_id'])

df_dup_filtered = df_dup_filtered.merge(
    df_unchanged_bvd[['bvd_id', 'Last avail. year']],
    on='bvd_id',  # Use 'on' since both DataFrames now have the same column name
    how='left',
)

# Group by 'National ID' and 'Name of the beneficiary' and keep the row with the highest 'Last avail. year'
df_highest_year = df_dup_filtered.loc[
    df_dup_filtered.groupby(['National ID', 'Name of the beneficiary'])['Last avail. year'].idxmax()
]

# Reset the index for the resulting DataFrame
df_highest_year = df_highest_year.reset_index(drop=True)

# Add a new column to store user input
df_highest_year['manual_similar'] = None

# Iterate through each row in the DataFrame
for index, row in df_highest_year.iterrows():
    # Print the names for comparison
    print(f"Name of the beneficiary: {row['Name of the beneficiary']}")
    print(f"Orbis name: {row['orbis_name']}")
    
    # Ask for user input
    user_input = input("Do you find the names similar? (y/n): ").strip().lower()
    
    # Validate input
    while user_input not in ['y', 'n']:
        print("Invalid input. Please enter 'y' for yes or 'n' for no.")
        user_input = input("Do you find the names similar? (y/n): ").strip().lower()
    
    # Append the input to the 'similar' column
    df_highest_year.at[index, 'similar'] = user_input

# Drop rows where the 'similar' column has 'n'
df_highest_year = df_highest_year[df_highest_year['manual_similar'] != 'n']

# Drop the 'similar' column if you no longer need it
#df_highest_year.drop(columns=['manual_similar'], inplace=True)

# Display the filtered DataFrame
print("Filtered DataFrame:")
print(df_highest_year)

In [33]:
# Step 1: Filter out duplicates from df
# Keep only rows that are not duplicated based on 'National ID' and 'Name of the beneficiary'
df_unique = df[~df.duplicated(subset=['National ID', 'Name of the beneficiary'], keep=False)].copy()

# Step 2: Append df_highest_year to the filtered df
# Ensure df_highest_year has the same columns as df
df_combined = pd.concat([df_unique, df_highest_year], ignore_index=True)

# Step 3: (Optional) Reset the index for the combined DataFrame
df_combined = df_combined.reset_index(drop=True)

df_combined.drop(columns=['manual_similar', 'Unchanged', 'Last avail. year'], inplace=True)
# Display the resulting DataFrame
print(df_combined)

           Ref-no.                           National ID  \
0      TM-10132031                           HRB 2572 NP   
1      TM-10132034                          HRB 10826 FF   
2      TM-10132040                           HRB 8957 NP   
3      TM-10130227                         HRB  10322 FL   
4      TM-10132036                           HRA 3469 CB   
...            ...                                   ...   
85678  TM-11018451        PIPE Kenan Nuradinovic | 47167   
85679  TM-12085423            RegionNuts2021.DE219288818   
85680  TM-11035893         Reisebüro Canan Hecht | 84032   
85681  TM-11052816                   Sinan Özden | 23568   
85682  TM-11656853  Verein Ensemble Resonanz GbR | 20359   

                        processed_nid          orbis_tradeno  orbis_vatno  \
0                             HRB2572     HRB2572NPNeuruppin                
1                            HRB10826  HRB10826FrankfurtOder  DE814592984   
2                             HRB8957       HRB8

In [34]:
df_combined.to_parquet(os.path.join(data_path, 'found_GER.parquet'), index=False)

# Cleaning: Size, Sector, and Region

In [ ]:
df = pd.read_csv(os.path.join(data_path, "merged_with_bvd_id.csv"))

In [ ]:
'''
# Get BvD ID for Orbis
df_bvd = df['BvD ID number']
output_bvd_path = os.path.join(file_path, 'BvD_ID.csv')
df_bvd.to_csv(output_bvd_path, index=False)
'''

In [7]:
#distinct_regions = pd.DataFrame(df['Region'].unique(), columns=['Region'])
#print(distinct_regions)

In [ ]:
region_to_state ={
    'Baden-Wuerttemberg' : ['baden-württemberg'],
    'Bayern' : ['bayern'],
    'Berlin' : ['berlin'],
    'Brandenburg' : ['brandenburg'],
    'Bremen' : ['bremen'],
    'Hamburg' : ['hamburg'],
    'Hessen' : ['hessen'],
    'Mecklenburg-Vorpommern' : ['mecklenburg-vorpommern'],
    'Niedersachsen' : ['niedersachsen'],
    'Nordrhein-Westfalen' : ['nordrhein-westfalen'],
    'Rheinland-Pfalz' : ['rheinland-pfalz'],
    'Saarland' : ['saarland'],
    'Sachsen' : ['sachsen'],
    'Sachsen-Anhalt' : ['sachsen-anhalt'],
    'Schleswig-Holstein' : ['schleswig-holstein'],
    'Thueringen' : ['thüringen']  
}

# Custom function to map 'Region' to 'State'
def map_region_to_state(region):
    region_lower = region.lower()
    for state, regions in region_to_state.items():
        if region_lower in regions:
            return state
    return

'''
# Create the 'State' column by applying the custom function
distinct_regions['State'] = distinct_regions['Region'].apply(map_region_to_state)

# Display the resulting DataFrame
print(distinct_regions)
'''

In [ ]:
'''
# Merge data from Orbis
prefix_ssr = 'StateAid_Size_Sector_Region_'

# List all Excel files in the directory with the specified prefix
xlsx_files = [f for f in os.listdir(file_path) if f.startswith(prefix_ssr) and f.endswith('.xlsx')]

# Initialize an empty list to store DataFrames
dataframes = []

# Process each Excel file
for xlsx_file in xlsx_files:
    # Load the Excel file
    excel_path = os.path.join(file_path, xlsx_file)
    xls = pd.ExcelFile(excel_path)
    
    # Check if the "Results" sheet exists, otherwise use the first sheet
    sheet_name = 'Results' if 'Results' in xls.sheet_names else xls.sheet_names[0]
    
    # Load the sheet into a DataFrame
    df = pd.read_excel(excel_path, sheet_name=sheet_name)
    
    #Remove the first column
    #df = df.iloc[:, 1:]

    # Append the DataFrame to the list
    dataframes.append(df)

# Merge all the DataFrames into a single DataFrame
merged_df = pd.concat(dataframes, ignore_index=True)

# Define the output Excel file path
output_file_path = os.path.join(file_path, 'Merged_StateAid_Size_Sector_Region.xlsx')  # Replace with your desired output file path

# Save the merged DataFrame to a new Excel file
merged_df.to_excel(output_file_path, index=False)

print(f"Merged Excel file has been saved at: {output_file_path}")
'''

In [10]:
df_orbis = pd.read_excel(os.path.join(file_path, "Merged_StateAid_Size_Sector_Region.xlsx"), dtype={'NACE Rev. 2, core code (4 digits)': str})

In [ ]:
counts = df_orbis['Type of region in country'].value_counts()
print(counts)

In [12]:
# Rename the first column to 'Number'
df_orbis.rename(columns={df_orbis.columns[0]: 'Number'}, inplace=True)

# Remove rows that have missing values in the 'Number' column
df_orbis = df_orbis.dropna(subset=['Number'])

In [13]:
#df_orbis = df_orbis[df_orbis['Type of region in country'].isna()]

In [14]:
merged_df = df.merge(df_orbis, on='BvD ID number', how='left')

In [15]:
# Map Region to State
merged_df['State'] = merged_df['Region'].apply(map_region_to_state)
# Map Region in country to Country
merged_df['State'] = merged_df['State'].fillna(merged_df['Region in country'])

In [ ]:
# Map cities in 'National ID' to State
# Germany in Region: Stendal, München, Ludwigshafen, Mainz, Jena, Dortmund, Bochum, Dresden, Bonn
# Dictionary for 'City Latin Alphabet' to 'State'
city_to_state = {
    'Stendal': 'Sachsen-Anhalt',
    'München': 'Bayern',
    'Ludwigshafen': 'Rheinland-Pfalz',
    'Mainz': 'Rheinland-Pfalz',
    'Jena': 'Thueringen',
    'Dortmund': 'Nordrhein-Westfalen',
    'Bochum': 'Nordrhein-Westfalen',
    'Dresden': 'Sachsen',
    'Bonn': 'Nordrhein-Westfalen'
}

# Function to map city to state
def map_city_to_state(national_id):
    for city, state in city_to_state.items():
        if city.lower() in national_id.lower():
            return state
    return None

city_nationalid = merged_df[(merged_df['Region'] == 'Germany') & (merged_df['State'].isna())]
city_nationalid['State'] = city_nationalid['National ID'].apply(map_city_to_state)
merged_df.loc[city_nationalid.index, 'State'] = city_nationalid['State']

In [17]:
# Convert to str
merged_df['Region'] = merged_df['Region'].astype(str)
merged_df['City\nLatin Alphabet'] = merged_df['City\nLatin Alphabet'].astype(str)
merged_df['City\nLocal Alphabet'] = merged_df['City\nLocal Alphabet'].astype(str)

In [ ]:
filtered_rows = merged_df[merged_df['State'].isna()]

unique_cities = filtered_rows['Region'].unique()
#print(unique_cities)

# More dictionary for city to state
additional_city_to_state = {
    'Unterfranken': 'Bayern',
    'Karlsruhe': 'Baden-Wuerttemberg',
    'Freising': 'Bayern',
    'Luebeck, Kreisfreie Stadt': 'Schleswig-Holstein',
    'Berchtesgadener Land': 'Bayern',
    'Lueneburg': 'Niedersachsen',
    'Oberbayern': 'Bayern',
    'Muenster': 'Nordrhein-Westfalen',
    'Hannover': 'Niedersachsen',
    'Stuttgart': 'Baden-Wuerttemberg',
    'Darmstadt': 'Hessen',
    'Niederbayern': 'Bayern',
    'Muenchen, Kreisfreie Stadt': 'Bayern',
    'Ostholstein': 'Schleswig-Holstein',
    'Koeln': 'Nordrhein-Westfalen',
    'Erzgebirgskreis': 'Sachsen',
    'Segeberg': 'Schleswig-Holstein',
    'Detmold': 'Nordrhein-Westfalen',
    'Leipzig': 'Sachsen',
    'Saarlouis': 'Saarland',
    'Duesseldorf': 'Nordrhein-Westfalen',
    'Braunschweig': 'Niedersachsen',
    'Koblenz': 'Rheinland-Pfalz',
    'Weser-Ems': 'Niedersachsen',
    'Rheinhessen-Pfalz': 'Rheinland-Pfalz',
    'Soest': 'Nordrhein-Westfalen',
    'Aurich': 'Niedersachsen',
    'Bonn, Kreisfreie Stadt': 'Nordrhein-Westfalen',
    'Bautzen': 'Sachsen',
    'Schwaben': 'Bayern',
    'Starnberg': 'Bayern',
    'Loerrach': 'Baden-Wuerttemberg',
    'Muenchen, Landkreis': 'Bayern',
    'Havelland': 'Brandenburg',
    'Helmstedt': 'Niedersachsen',
    'Dresden': 'Sachsen',
    'Ortenaukreis': 'Baden-Wuerttemberg',
    'Stuttgart, Stadtkreis': 'Baden-Wuerttemberg',
    'Karlsruhe, Stadtkreis': 'Baden-Wuerttemberg',
    'Hohenlohekreis': 'Baden-Wuerttemberg',
    'Rottal-Inn': 'Bayern',
    'Recklinghausen': 'Nordrhein-Westfalen',
    'Emden, Kreisfreie Stadt': 'Niedersachsen',
    'Region Hannover': 'Niedersachsen',
    'Gelsenkirchen, Kreisfreie Stadt': 'Nordrhein-Westfalen',
    'Ludwigshafen am Rhein, Kreisfreie Stadt': 'Rheinland-Pfalz',
    'Freiburg im Breisgau, Stadtkreis': 'Baden-Wuerttemberg',
    'Duesseldorf, Kreisfreie Stadt': 'Nordrhein-Westfalen',
    'Paderborn': 'Nordrhein-Westfalen',
    'Neuburg-Schrobenhausen': 'Bayern',
    'Main-Kinzig-Kreis': 'Hessen',
    'Hof, Landkreis': 'Bayern',
    'Essen': 'Nordrhein-Westfalen',
    'Duisburg, Kreisfreie Stadt': 'Nordrhein-Westfalen',
    'Bochum, Kreisfreie Stadt': 'Nordrhein-Westfalen',
    'Guetersloh': 'Nordrhein-Westfalen',
    'Wartburgkreis': 'Thueringen',
    'Karlsruhe, Landkreis': 'Baden-Wuerttemberg',
    'Ravensburg': 'Baden-Wuerttemberg',
    'Saale-Orla-Kreis': 'Thueringen',
    'Jena, Kreisfreie Stadt': 'Thueringen',
    'Vorpommern-Greifswald': 'Mecklenburg-Vorpommern',
    'Wesel': 'Nordrhein-Westfalen',
    'Oberpfalz': 'Bayern',
    'Freyung-Grafenau': 'Bayern',
    'Ansbach, Landkreis': 'Bayern',
    'Frankfurt am Main, Kreisfreie Stadt': 'Hessen',
    'Tuebingen': 'Baden-Wuerttemberg',
    'Arnsberg': 'Nordrhein-Westfalen',
    'Freiburg': 'Baden-Wuerttemberg',
    'Kassel': 'Hessen',
    'Chemnitz': 'Sachsen',
    'Mainz-Bingen': 'Rheinland-Pfalz',
    'Coesfeld': 'Nordrhein-Westfalen',
    'Heilbronn, Landkreis': 'Baden-Wuerttemberg',
    'Kleve': 'Nordrhein-Westfalen',
    'Saalekreis': 'Sachsen-Anhalt',
    'Weilheim-Schongau': 'Bayern',
    'Marburg-Biedenkopf': 'Hessen',
    'Mittelfranken': 'Bayern',
    'Neckar-Odenwald-Kreis': 'Baden-Wuerttemberg',
    'Biberach': 'Baden-Wuerttemberg',
    'Altoetting': 'Bayern',
    'Bad Kreuznach': 'Rheinland-Pfalz',
    'Muenchen' : 'Bayern',
    'GTTTINGEN' : 'Niedersachsen',
    'MAGDEBURG' : 'Sachsen-Anhalt',
    'Stuttgart' : 'Baden-Wuerttemberg',
    'Hannover' : 'Niedersachsen',
    'Münster' : 'Nordrhein-Westfalen',
    'Oberbayern' : 'Bayern',
    'AMBERG' : 'Bayern',
    'Altötting' : 'Bayern',
    "Köln" : 'Nordrhein-Westfalen',
    'Düsseldorf' : 'Nordrhein-Westfalen',
    'Lüneburg' : 'Niedersachsen',
    'Lübeck' : 'Schleswig-Holstein',
    'Lörrach' : 'Baden-Wuerttemberg',
    'Gütersloh' : 'Nordrhein-Westfalen',
    'Tübingen' : 'Baden-Wuerttemberg',
    'MSNCHEN' : 'Bayern',
    'Berlin' : 'Berlin',
    'BAD HOMBURG' : 'Hessen',
    'DÜSSELDOPRF' : 'Nordrhein-Westfalen',
    'asperg' : 'Baden-Wuerttemberg',
    'hagen' : 'Nordrhein-Westfalen',
    'oberstdorf' : 'Bayern',
    'MANNHEIM' : 'Baden-Wuerttemberg',
    'ROSENHEIM' : 'Bayern',
    'HALLSTADT' : 'Bayern',
    'KAISERSLAUTERN' : 'Rheinland-Pfalz',
    'hagen' : 'Nordrhein-Westfalen',
    'bremen' : 'Bremen',
    'NÜRNBERG' : 'Bayern',
    'bismark' : 'Sachsen-Anhalt',
    'SCHONAU AM KONIGSSEE' : 'Bayern',
    'MÜLHEIM' : 'Nordrhein-Westfalen',
    'AACHEN' : 'Nordrhein-Westfalen',
    'WEITERSTADT' : 'Hessen',
    'LIMBACH-OBERFROHNA' : 'Sachsen',
    'MUNCHEN': 'Bayern',
    'WARTMANNSROTH': 'Bayern',
    'HÖXTER': 'Nordrhein-Westfalen',
    'WESTERSTEDE': 'Niedersachsen',
    'TRIER': 'Rheinland-Pfalz',
    'OSTBEVERN': 'Nordrhein-Westfalen',
    'LUDWIGSBURG': 'Baden-Wuerttemberg',
    'LANGELSHEIM': 'Niedersachsen',
    'HOCHHEIM AM MAIN': 'Hessen',
    'HIDDENHAUSEN': 'Nordrhein-Westfalen',
    'GRÄFENRODA': 'Thueringen',
    'GERATAL': 'Thueringen',
    'ERLANGEN': 'Bayern',
    'EINBECK': 'Niedersachsen',
    'DSSSELDORF': 'Nordrhein-Westfalen'
}

city_to_state.update(additional_city_to_state)

In [ ]:
# Filter rows where 'State' is missing and 'Region'
filtered_rows = merged_df[merged_df['State'].isna()]

# Apply the city to state mapping function to the 'City Latin Alphabet' column
filtered_rows['State'] = filtered_rows['Region'].apply(map_city_to_state)

# Fill in the 'State' column in the original DataFrame
merged_df.loc[filtered_rows.index, 'State'] = filtered_rows['State']

In [ ]:
# Filter rows where 'State' is missing and 'Region' is 'Germany'
filtered_rows = merged_df[(merged_df['State'].isna()) & (merged_df['Region'] == 'Germany')]

# Apply the city to state mapping function to the 'City Latin Alphabet' column
filtered_rows['State'] = filtered_rows['City\nLocal Alphabet'].apply(map_city_to_state)

# Fill in the 'State' column in the original DataFrame
merged_df.loc[filtered_rows.index, 'State'] = filtered_rows['State']

In [21]:
# Rename "NACE Rev. 2, core code (4 digits)" to "nace_code"
merged_df.rename(columns={'NACE Rev. 2, core code (4 digits)': 'nace_code'}, inplace=True)
merged_df['nace_code'] = merged_df['nace_code'].astype(str)

In [ ]:
filtered_rows = merged_df[merged_df['nace_code'].isna()]

missing_sector_to_nacecode = {
    'Wired telecommunications activities': '6110',
    'Manufacture of other fabricated metal products n.e.c.': '2599',
    'Electric power generation, transmission and distribution': '3511',
    'Manufacture of basic iron and steel and of ferro-alloys': '2410',
    'Hotels and similar accommodation': '5510',
    'Repair and maintenance of other transport equipment': '3317',
    'Travel agency and tour operator activities': '7911',
    'Wholesale of agricultural machinery, equipment and supplies': '4661',
    'Other specialised construction activities n.e.c.': '4399',
    'Retail sale of furniture, lighting equipment and other household articles in specialised stores': '4759',
    'Wholesale and retail trade and repair of motor vehicles and motorcycles': '4510',
    'Retail sale of sporting equipment in specialised stores': '4764',
    'Manufacture of metal structures and parts of structures': '2512',
    'Retail sale of clothing in specialised stores': '4771',
    'Water transport': '5040',
    'Management of real estate on a fee or contract basis': '6832',
    'Manufacture of other chemical products': '2059',
    'WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES': '4500',
    'Manufacture of lifting and handling equipment': '2822',
    'Manufacture of basic metals': '2400', 
    'Manufacture of doors and windows of metal': '2512',
    'Scientific research and development': '7211',
    'Raising of swine/pigs': '0146',
    'Restaurants and mobile food service activities': '5610',
    'Crop and animal production, hunting and related service activities': '0100',
    'Mixed farming': '0150',
    'Raising of dairy cattle': '0141',
    'Manufacture of prepared pet foods': '1092',
    'Other research and experimental development on natural sciences and engineering': '7219',
    'Manufacture of vegetable and animal oils and fats': '1041',
    'Wholesale of grain, unmanufactured tobacco, seeds and animal feeds': '4621',
    'Production of meat and poultry meat products': '1013',
    'Retail sale via mail order houses or via Internet': '4791',
    'Animal production': '0140',
}

def map_sector_to_nacecode(sector):
    for sector_name, nace_code in missing_sector_to_nacecode.items():
        if sector_name.lower() in sector.lower():
            return nace_code
    return None

# Apply to missing values
filtered_rows['nace_code'] = filtered_rows['Sector (NACE)'].apply(map_sector_to_nacecode)
merged_df.loc[filtered_rows.index, 'nace_code'] = filtered_rows['nace_code']

In [23]:
# Add '0' to rows with 3 digits in 'nace_code'
merged_df['nace_code'] = merged_df['nace_code'].apply(lambda x: x.zfill(4) if len(x) == 3 else x)

In [ ]:
# Correct nace code of 0100, 1000, and 1010, and one specific company with 9810
filtered_rows = merged_df[merged_df['nace_code'].str.contains(r'^(0100|1000|1010|9810)$', na=False)]
filtered_rows['nace_code'] = filtered_rows['Sector (NACE)'].apply(map_sector_to_nacecode)
merged_df.loc[filtered_rows.index, 'nace_code'] = filtered_rows['nace_code']

In [25]:
#print(merged_df['NACE Rev. 2, core code (4 digits)'].unique())

In [26]:
# Drop rows without 'State' and Aid amount
merged_df['no_aid_amount'] = merged_df['Aid element, expressed as full amount'].apply(lambda x: 1 if any(char in str(x) for char in ['>', '<', '-']) else 0)
merged_df = merged_df.dropna(subset=['State', 'Aid element, expressed as full amount'])
merged_df = merged_df[merged_df['no_aid_amount'] == 0]

In [27]:
# Define the file path for the NoEmp.xlsx file
noemp_file_path = os.path.join(file_path, "NoEmp.xlsx")

# Import the specific sheet "Result" from the NoEmp.xlsx file
df_noemp = pd.read_excel(noemp_file_path, sheet_name="Results")

# Rename column 'Number of employees\nLast avail. yr' to 'NumEmp'
df_noemp.rename(columns={'Number of employees\nLast avail. yr': 'NumEmp'}, inplace=True)

# Merge the 'Number of employees Last avail. yr' column from df_noemp into merged_df based on 'BvD ID number'
merged_df = merged_df.merge(df_noemp[['BvD ID number', 'NumEmp']], on='BvD ID number', how='left')

In [28]:
# Export
output_path = os.path.join(file_path, 'with_aid_state_5.csv')
merged_df.to_csv(output_path, index=False)

# Descriptive Stats: Size, Sector, and Region

In [ ]:
# Load file
df = pd.read_csv(os.path.join(file_path, 'with_aid_state_5.csv'),dtype={'nace_code': str})

In [ ]:
# Dataset with unique firms
df_nodup = df.drop_duplicates(subset='BvD ID number'), keep='first')

In [31]:
# Replace "n.a." with NaN
df_nodup['NumEmp'] = df_nodup['NumEmp'].replace("n.a.", np.nan)

# Convert the 'NumEmp' column to numeric
df_nodup['NumEmp'] = pd.to_numeric(df_nodup['NumEmp'], errors='coerce')

# Drop rows with missing values in the 'NumEmp' column
df_nodup = df_nodup.dropna(subset=['NumEmp'])

# Define the bins and labels for the ranges
bins = [0, 9, 19, 49, 99, 249, float('inf')]
labels = ['1-9', '10-19', '20-49', '50-99', '100-249', '250+']

# Classify firms into the specified ranges
df_nodup['Firm Size'] = pd.cut(df_nodup['NumEmp'], bins=bins, labels=labels, right=True)

# Count the number of firms in each range
firm_size_distribution = df_nodup['Firm Size'].value_counts().sort_index().reset_index()
firm_size_distribution.columns = ['Firm Size', 'Firm Count']

# Count the number of firms with missing values in 'NumEmp'
missing_count = df_nodup['NumEmp'].isna().sum()
missing_row = pd.DataFrame([['Missing', missing_count]], columns=['Firm Size', 'Firm Count'])

# Append the missing row to the distribution table
firm_size_distribution = pd.concat([firm_size_distribution, missing_row], ignore_index=True)

# Calculate the percentage of firm count
total_firms = firm_size_distribution['Firm Count'].sum()
firm_size_distribution['Percentage'] = (firm_size_distribution['Firm Count'] / total_firms) * 100

# Round the percentage to 2 digits after the decimal separator
firm_size_distribution['Percentage'] = firm_size_distribution['Percentage'].round(2)

# Custom formatting function to display "<0.01" for values equal to 0.00
def format_percentage(value):
    return "<0.01" if value == 0.00 else f"{value:.2f}"

# Apply the custom formatting function to the 'Percentage' column
firm_size_distribution['Percentage'] = firm_size_distribution['Percentage'].apply(format_percentage)

# Add a final row to sum all the firm counts to get the total number of firms
total_row = pd.DataFrame([['Total', total_firms, '100.00']], columns=['Firm Size', 'Firm Count', 'Percentage'])
firm_size_distribution = pd.concat([firm_size_distribution, total_row], ignore_index=True)

# Display the resulting table
print(firm_size_distribution)

# Latex
latex_table = firm_size_distribution.to_latex(index=False)
print(latex_table)

  Firm Size  Firm Count Percentage
0       1-9       14894      27.02
1     10-19       10969      19.90
2     20-49       14688      26.65
3     50-99        6387      11.59
4   100-249        4264       7.74
5      250+        3911       7.10
6   Missing           0      <0.01
7     Total       55113     100.00
\begin{tabular}{lrl}
\toprule
Firm Size & Firm Count & Percentage \\
\midrule
1-9 & 14894 & 27.02 \\
10-19 & 10969 & 19.90 \\
20-49 & 14688 & 26.65 \\
50-99 & 6387 & 11.59 \\
100-249 & 4264 & 7.74 \\
250+ & 3911 & 7.10 \\
Missing & 0 & <0.01 \\
Total & 55113 & 100.00 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_110029/1675525726.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nodup['NumEmp'] = df_nodup['NumEmp'].replace("n.a.", np.nan)
/tmp/ipykernel_110029/1675525726.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nodup['NumEmp'] = pd.to_numeric(df_nodup['NumEmp'], errors='coerce')


In [32]:
# Create a mapping dictionary for the first two digits of the nace_code to sector descriptions
nace_sector_mapping = {
    '01': 'A: AGRICULTURE, FORESTRY AND FISHING',
    '02': 'A: AGRICULTURE, FORESTRY AND FISHING',
    '03': 'A: AGRICULTURE, FORESTRY AND FISHING',
    '05': 'B: MINING AND QUARRYING',
    '06': 'B: MINING AND QUARRYING',
    '07': 'B: MINING AND QUARRYING',
    '08': 'B: MINING AND QUARRYING',
    '09': 'B: MINING AND QUARRYING',
    '10': 'C: MANUFACTURING',
    '11': 'C: MANUFACTURING',
    '12': 'C: MANUFACTURING',
    '13': 'C: MANUFACTURING',
    '14': 'C: MANUFACTURING',
    '15': 'C: MANUFACTURING',
    '16': 'C: MANUFACTURING',
    '17': 'C: MANUFACTURING',
    '18': 'C: MANUFACTURING',
    '19': 'C: MANUFACTURING',
    '20': 'C: MANUFACTURING',
    '21': 'C: MANUFACTURING',
    '22': 'C: MANUFACTURING',
    '23': 'C: MANUFACTURING',
    '24': 'C: MANUFACTURING',
    '25': 'C: MANUFACTURING',
    '26': 'C: MANUFACTURING',
    '27': 'C: MANUFACTURING',
    '28': 'C: MANUFACTURING',
    '29': 'C: MANUFACTURING',
    '30': 'C: MANUFACTURING',
    '31': 'C: MANUFACTURING',
    '32': 'C: MANUFACTURING',
    '33': 'C: MANUFACTURING',
    '35': 'D: ELECTRICITY, GAS, STEAM AND AIR CONDITIONING SUPPLY',
    '36': 'E: WATER SUPPLY; SEWERAGE, WASTE MANAGEMENT AND REMEDIATION ACTIVITIES',
    '37': 'E: WATER SUPPLY; SEWERAGE, WASTE MANAGEMENT AND REMEDIATION ACTIVITIES',
    '38': 'E: WATER SUPPLY; SEWERAGE, WASTE MANAGEMENT AND REMEDIATION ACTIVITIES',
    '39': 'E: WATER SUPPLY; SEWERAGE, WASTE MANAGEMENT AND REMEDIATION ACTIVITIES',
    '41': 'F: CONSTRUCTION',
    '42': 'F: CONSTRUCTION',
    '43': 'F: CONSTRUCTION',
    '45': 'G: WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES',
    '46': 'G: WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES',
    '47': 'G: WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES',
    '49': 'H: TRANSPORTATION AND STORAGE',
    '50': 'H: TRANSPORTATION AND STORAGE',
    '51': 'H: TRANSPORTATION AND STORAGE',
    '52': 'H: TRANSPORTATION AND STORAGE',
    '53': 'H: TRANSPORTATION AND STORAGE',
    '55': 'I: ACCOMMODATION AND FOOD SERVICE ACTIVITIES',
    '56': 'I: ACCOMMODATION AND FOOD SERVICE ACTIVITIES',
    '58': 'J: INFORMATION AND COMMUNICATION',
    '59': 'J: INFORMATION AND COMMUNICATION',
    '60': 'J: INFORMATION AND COMMUNICATION',
    '61': 'J: INFORMATION AND COMMUNICATION',
    '62': 'J: INFORMATION AND COMMUNICATION',
    '63': 'J: INFORMATION AND COMMUNICATION',
    '64': 'K: FINANCIAL AND INSURANCE ACTIVITIES',
    '65': 'K: FINANCIAL AND INSURANCE ACTIVITIES',
    '66': 'K: FINANCIAL AND INSURANCE ACTIVITIES',
    '68': 'L: REAL ESTATE ACTIVITIES',
    '69': 'M: PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES',
    '70': 'M: PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES',
    '71': 'M: PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES',
    '72': 'M: PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES',
    '73': 'M: PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES',
    '74': 'M: PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES',
    '75': 'M: PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES',
    '77': 'N: ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES',
    '78': 'N: ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES',
    '79': 'N: ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES',
    '80': 'N: ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES',
    '81': 'N: ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES',
    '82': 'N: ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES',
    '84': 'O: PUBLIC ADMINISTRATION AND DEFENCE; COMPULSORY SOCIAL SECURITY',
    '85': 'P: EDUCATION',
    '86': 'Q: HUMAN HEALTH AND SOCIAL WORK ACTIVITIES',
    '87': 'Q: HUMAN HEALTH AND SOCIAL WORK ACTIVITIES',
    '88': 'Q: HUMAN HEALTH AND SOCIAL WORK ACTIVITIES',
    '90': 'R: ARTS, ENTERTAINMENT AND RECREATION',
    '91': 'R: ARTS, ENTERTAINMENT AND RECREATION',
    '92': 'R: ARTS, ENTERTAINMENT AND RECREATION',
    '93': 'R: ARTS, ENTERTAINMENT AND RECREATION',
    '94': 'S: OTHER SERVICE ACTIVITIES',
    '95': 'S: OTHER SERVICE ACTIVITIES',
    '96': 'S: OTHER SERVICE ACTIVITIES',
    '97': 'T: ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS; UNDIFFERENTIATED GOODS- AND SERVICES-PRODUCING ACTIVITIES OF HOUSEHOLDS FOR OWN USE',
    '98': 'T: ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS; UNDIFFERENTIATED GOODS- AND SERVICES-PRODUCING ACTIVITIES OF HOUSEHOLDS FOR OWN USE',
    '99': 'U: ACTIVITIES OF EXTRATERRITORIAL ORGANIZATIONS AND BODIES'
}

#Extract the first two digits of the 'nace_code' to determine the sector
df_nodup['sector_code'] = df_nodup['nace_code'].str[:2]

# Map the sectors using the mapping dictionary
df_nodup['sector'] = df_nodup['sector_code'].map(nace_sector_mapping)

# Group by sector to count the number of firms in each sector
sector_distribution = df_nodup['sector'].value_counts().reset_index()
sector_distribution.columns = ['Sector', 'Firm Count']

# Calculate the percentage of firm count
total_firms = sector_distribution['Firm Count'].sum()
sector_distribution['Percentage'] = (sector_distribution['Firm Count'] / total_firms) * 100

# Round the percentage to 2 digits after the decimal separator
sector_distribution['Percentage'] = sector_distribution['Percentage'].round(2)

# Custom formatting function to display "<0.01" for values equal to 0.00
def format_percentage(value):
    return "<0.01" if value == 0.00 else f"{value:.2f}"

# Apply the custom formatting function to the 'Percentage' column
sector_distribution['Percentage'] = sector_distribution['Percentage'].apply(format_percentage)

# Sort the table by the sector codes in alphabetical order
sector_distribution = sector_distribution.sort_values(by='Sector')

# Add a final row to sum all the firm counts to get the total number of firms
total_row = pd.DataFrame([['Total', total_firms, '100.00']], columns=['Sector', 'Firm Count', 'Percentage'])
sector_distribution = pd.concat([sector_distribution, total_row], ignore_index=True)


# Convert the DataFrame to a LaTeX tabularx environment
latex_table = sector_distribution.to_latex(index=False)

# Display the resulting table
print(latex_table)

\begin{tabular}{lrl}
\toprule
Sector & Firm Count & Percentage \\
\midrule
A: AGRICULTURE, FORESTRY AND FISHING & 814 & 1.48 \\
B: MINING AND QUARRYING & 95 & 0.17 \\
C: MANUFACTURING & 12261 & 22.25 \\
D: ELECTRICITY, GAS, STEAM AND AIR CONDITIONING SUPPLY & 1023 & 1.86 \\
E: WATER SUPPLY; SEWERAGE, WASTE MANAGEMENT AND REMEDIATION ACTIVITIES & 417 & 0.76 \\
F: CONSTRUCTION & 2919 & 5.30 \\
G: WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES & 10389 & 18.85 \\
H: TRANSPORTATION AND STORAGE & 2603 & 4.72 \\
I: ACCOMMODATION AND FOOD SERVICE ACTIVITIES & 5594 & 10.15 \\
J: INFORMATION AND COMMUNICATION & 3011 & 5.46 \\
K: FINANCIAL AND INSURANCE ACTIVITIES & 776 & 1.41 \\
L: REAL ESTATE ACTIVITIES & 1225 & 2.22 \\
M: PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES & 5038 & 9.14 \\
N: ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES & 4003 & 7.26 \\
O: PUBLIC ADMINISTRATION AND DEFENCE; COMPULSORY SOCIAL SECURITY & 47 & 0.09 \\
P: EDUCATION & 421 & 0.76 \\
Q: HUMAN HEA

In [33]:
# Count the number of firms in each state
state_distribution = df_nodup['State'].value_counts().reset_index()
state_distribution.columns = ['State', 'Firm Count']

# Calculate the percentage of firm count
total_firms = state_distribution['Firm Count'].sum()
state_distribution['Percentage'] = (state_distribution['Firm Count'] / total_firms) * 100

# Round the percentage to 2 digits after the decimal separator
state_distribution['Percentage'] = state_distribution['Percentage'].round(2)

# Custom formatting function to display "<0.01" for values equal to 0.00
def format_percentage(value):
    return "<0.01" if value == 0.00 else f"{value:.2f}"

# Apply the custom formatting function to the 'Percentage' column
state_distribution['Percentage'] = state_distribution['Percentage'].apply(format_percentage)

# Sort the table by the number of firm counts in descending order
state_distribution = state_distribution.sort_values(by='Firm Count', ascending=False)

# Add a final row to sum all the firm counts to get the total number of firms
total_row = pd.DataFrame([['Total', total_firms, '100.00']], columns=['State', 'Firm Count', 'Percentage'])
state_distribution = pd.concat([state_distribution, total_row], ignore_index=True)

# Display the resulting table
print(state_distribution)

# Convert the DataFrame to a LaTeX tabular environment
latex_table = state_distribution.to_latex(index=False)
print(latex_table)

                     State  Firm Count Percentage
0      Nordrhein-Westfalen       13003      23.59
1                   Bayern       10898      19.77
2       Baden-Wuerttemberg        6756      12.26
3                   Hessen        4403       7.99
4            Niedersachsen        3601       6.53
5          Rheinland-Pfalz        2566       4.66
6                   Berlin        2483       4.51
7                  Hamburg        2069       3.75
8                  Sachsen        2043       3.71
9       Schleswig-Holstein        1520       2.76
10              Thueringen        1270       2.30
11          Sachsen-Anhalt        1254       2.28
12  Mecklenburg-Vorpommern        1120       2.03
13             Brandenburg        1111       2.02
14                Saarland         671       1.22
15                  Bremen         345       0.63
16                   Total       55113     100.00
\begin{tabular}{lrl}
\toprule
State & Firm Count & Percentage \\
\midrule
Nordrhein-Westfalen & 1300